In [23]:
import pandas as pd
import numpy as np
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform

In [2]:
#estaticos_market = pd.read_csv('~/codenation/projeto_final/files/estaticos_market.csv')
estaticos_market = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio1.csv')
print(estaticos_market.shape)

(555, 182)


In [ ]:
percent_missing = estaticos_market.isnull().sum() * 100 / len(estaticos_market)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing,
                                 'dtypes': estaticos_market.dtypes,})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df.head(181)

In [29]:
from scipy import sparse

train2 = estaticos_market
str_cols = train2.select_dtypes(include=['object']).columns
train2.loc[:, str_cols] = train2.loc[:, str_cols].fillna(train2['id'])
booleandf = train2.select_dtypes(include=[bool]).columns
booleanDictionary = {True: 'T', False: 'F'}
for column in booleandf:
    train2[column] = train2[column].map(booleanDictionary)
train2=  train2.astype(str)
objectdf  = train2.select_dtypes(include=[object]).columns
train2 = train2.replace(to_replace ='nan', value = train2['id'])
for column in objectdf:
    train2[column] = train2[column].str.replace(' ', '')
    train2[column] = np.where(train2[column] == 'nan', train2['id'], train2[column])
    train2[column] = column+train2[column]
train2 = train2.drop(['Unnamed: 0'],axis=1) 
train2 = train2.drop(['fl_matriz'],axis=1) 
indices2 = pd.Series(train2.index, index=train2['id']).drop_duplicates()
train2.drop(['id'],axis=1)
train2['new'] = train2.apply(''.join, axis=1)
train3 = train2['new']
del train2
cv = CountVectorizer()
count_matrix = cv.fit_transform(train3)
count_matrix = cv.transform(train3)
count_matrix = sparse.csr_matrix(count_matrix)
cosine_sim = pairwise_kernels(count_matrix, Y=None, metric='cosine', filter_params=False, n_jobs=5)
#cosine_sim = cosine_similarity(count_matrix)

#cosine_sim = linear_kernel(count_matrix,count_matrix)

#cosine_sim = distance.cosine(count_matrix.todense(),count_matrix.todense())

In [27]:
def get_recommendations2(id, cosine_sim=cosine_sim):
    try:
        idx = indices2[id]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        id_indices =[]
        for i in sim_scores:
            if i[1]>=0.5:
                id_indices.append(i[0]) 
        result=estaticos_market['id'].iloc[id_indices].values.tolist()
    except:
        result=([''])
    return result
def get_id2(idlist):
    data_day_list = []
    for element in idlist:
        data_day =(estaticos_market[estaticos_market.id==element])
        data_day_list.append(data_day)
    final_data_day = pd.concat(data_day_list)
    return final_data_day

In [30]:
estaticos_portfolio1 = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio1.csv')
estaticos_portfolio2 = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio2.csv')
estaticos_portfolio3 = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio3.csv')
#print(estaticos_portfolio1.shape)
#print(estaticos_portfolio2.shape)
#print(estaticos_portfolio3.shape)
requis=(['7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720','70485e6b8abe52d8fb4e0bf060fb6f0f4f8576cb7583d885a8dcac38f506389','6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570'])
result_list=[]
for element in requis:
    try:
        print(element)
        result = get_id2(get_recommendations2('id'+element))
        result_list.append(result)
    except NameError:
        print("Id "+element+" não encontrado!")
result_final = pd.concat(result_list)
result_final.drop_duplicates(keep = 'first', inplace = True)
result_final
result_list2 = []
print(requis)
for element in requis:
    result = (result_final[result_final.id==element]) 
    result_list2.append(result)
result_final2 = pd.concat(result_list2)
i1 = len(requis)
f1,f2 = result_final2.shape
print("Percentual de item solicitado no resultado: "+str(round(f1/i1*100,2))+'%')
result_final

7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720
70485e6b8abe52d8fb4e0bf060fb6f0f4f8576cb7583d885a8dcac38f506389
6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570
['7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720', '70485e6b8abe52d8fb4e0bf060fb6f0f4f8576cb7583d885a8dcac38f506389', '6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570']
Percentual de item solicitado no resultado: 66.67%


,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
9,9,7d58990ba786a3a26617a8afbf814a9919ab34aa09013a...,T,SOCIEDADE EMPRESARIA LIMITADA,PI,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,5.309589,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
37,37,d4cb950ffc7f6de0a38403a590bff553a03185b1ec9dac...,T,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,16.350685,15 a 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
426,426,6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c78...,T,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,SERVIÇO,6.361644,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
155,155,23be3d5e72271bf16f51c448c5e9b1287e2fa19818221b...,T,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,16.523288,15 a 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
133,133,2be489bf8b056c634f8bfd2367d09608b4421e5911fc12...,T,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,3.794521,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
536,536,5bf996b49298eb2fc838bc393d7a17a60d80647340028c...,T,SOCIEDADE EMPRESARIA LIMITADA,AC,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,6.917808,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
488,488,dc1827198539361b91487caaa8dc1ae44e9a898aec36c9...,F,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,26.145205,> 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
528,528,f66944b5f183ab1db5d2688fd960431a1cc467e62ac99a...,T,EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITAD...,AM,OUTROS,COMERCIO VAREJISTA,COMERCIO,0.953425,<= 1,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.000000,0
515,515,a910c2add4cb3ce618ca55334e2fe8187d7a85de394c8f...,T,EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITAD...,AC,OUTROS,COMERCIO VAREJISTA,COMERCIO,1.895890,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
353,353,d23652754eb2237240d1a4e4265e891cd109f25bd85d22...,T,SOCIEDADE EMPRESARIA LIMITADA,PI,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,10.073973,10 a 15,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.000000,0


In [ ]:
id='id6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570'
idx = indices2[id]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
print(sim_scores)
id_indices =[]
for i in sim_scores:
    if i[1]>=0.0:
        id_indices.append(i[0]) 
result=estaticos_market['id'].iloc[id_indices].values.tolist()